In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Load and clean data
def load_and_prepare_data(filepath):
    df = pd.read_csv(filepath)
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
    df = df.sort_values('Date').reset_index(drop=True)
    
    # Handle any missing values
    df['Price'] = df['Price'].interpolate(method='linear')
    
    # Calculate log returns
    df['Log_Return'] = np.log(df['Price']) - np.log(df['Price'].shift(1))
    df['Return'] = df['Price'].pct_change()
    
    return df

# Visualize time series
def plot_price_series(df):
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Raw price series
    axes[0].plot(df['Date'], df['Price'], color='steelblue', linewidth=1)
    axes[0].set_title('Brent Crude Oil Prices (1987-2022)', fontsize=14)
    axes[0].set_ylabel('Price (USD/barrel)')
    axes[0].grid(True, alpha=0.3)
    
    # Log returns
    axes[1].plot(df['Date'], df['Log_Return'], color='coral', linewidth=0.5)
    axes[1].set_title('Daily Log Returns', fontsize=14)
    axes[1].set_ylabel('Log Return')
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
    axes[1].grid(True, alpha=0.3)
    
    # Volatility (rolling standard deviation)
    df['Volatility'] = df['Log_Return'].rolling(window=30).std() * np.sqrt(252)
    axes[2].plot(df['Date'], df['Volatility'], color='purple', linewidth=1)
    axes[2].set_title('30-Day Rolling Volatility', fontsize=14)
    axes[2].set_ylabel('Annualized Volatility')
    axes[2].set_xlabel('Date')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('price_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def bayesian_change_point_model(data, n_changepoints=5):
    """
    Bayesian multiple change point model for oil prices
    """
    with pm.Model() as model:
        # Data
        prices = data['Price'].values
        n_obs = len(prices)
        
        # Priors for change points
        tau = pm.DiscreteUniform('tau', 
                                 lower=0, 
                                 upper=n_obs-1, 
                                 shape=n_changepoints)
        
        # Sort change points
        tau_sorted = pm.Deterministic('tau_sorted', 
                                      pm.math.sort(tau))
        
        # Segment means and standard deviations
        segment_means = []
        segment_sigmas = []
        
        for i in range(n_changepoints + 1):
            if i == 0:
                start_idx = 0
            else:
                start_idx = tau_sorted[i-1]
            
            if i == n_changepoints:
                end_idx = n_obs
            else:
                end_idx = tau_sorted[i]
            
            # Priors for segment parameters
            mu = pm.Normal(f'mu_{i}', mu=np.mean(prices), sigma=np.std(prices)*2)
            sigma = pm.HalfNormal(f'sigma_{i}', sigma=np.std(prices))
            
            segment_means.append(mu)
            segment_sigmas.append(sigma)
        
        # Likelihood using switch function
        idx = np.arange(n_obs)
        mu_combined = segment_means[0]
        sigma_combined = segment_sigmas[0]
        
        for i in range(1, n_changepoints + 1):
            mask = idx >= tau_sorted[i-1]
            mu_combined = pm.math.switch(mask, segment_means[i], mu_combined)
            sigma_combined = pm.math.switch(mask, segment_sigmas[i], sigma_combined)
        
        # Likelihood
        likelihood = pm.Normal('likelihood', 
                               mu=mu_combined, 
                               sigma=sigma_combined, 
                               observed=prices)
    
    return model

def run_mcmc(model, draws=3000, tune=1000):
    """
    Run MCMC sampling
    """
    with model:
        # Sampling
        trace = pm.sample(draws=draws, 
                         tune=tune, 
                         chains=4, 
                         cores=4,
                         return_inferencedata=True,
                         progressbar=True)
    
    return trace

In [ ]:
def analyze_results(trace, data, events_df):
    """
    Analyze and visualize MCMC results
    """
    # Convergence diagnostics
    summary = az.summary(trace, var_names=['tau', 'mu_', 'sigma_'])
    print("Model Summary:")
    print(summary)
    
    # Trace plots
    az.plot_trace(trace, var_names=['tau'])
    plt.suptitle('MCMC Trace Plots for Change Points', y=1.02)
    plt.tight_layout()
    plt.savefig('trace_plots.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Posterior distributions of change points
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, tau_var in enumerate(['tau_sorted[0]', 'tau_sorted[1]', 'tau_sorted[2]', 
                                 'tau_sorted[3]', 'tau_sorted[4]']):
        if tau_var in trace.posterior:
            tau_samples = trace.posterior[tau_var].values.flatten()
            
            # Convert to dates
            tau_dates = data['Date'].iloc[tau_samples.astype(int)].values
            
            axes[i].hist(tau_dates, bins=50, alpha=0.7, color='steelblue')
            axes[i].set_title(f'Change Point {i+1}')
            axes[i].set_xlabel('Date')
            axes[i].set_ylabel('Frequency')
            
            # Add event markers
            for _, event in events_df.iterrows():
                event_date = pd.to_datetime(event['Date'])
                axes[i].axvline(x=event_date, color='red', alpha=0.5, linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    plt.savefig('change_point_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return summary

def quantify_impacts(trace, data, change_point_idx):
    """
    Quantify price impacts before and after change points
    """
    results = []
    
    for i, cp_idx in enumerate(change_point_idx):
        # Get samples for segment parameters
        mu_before = trace.posterior[f'mu_{i}'].values.flatten()
        mu_after = trace.posterior[f'mu_{i+1}'].values.flatten()
        
        # Calculate impact metrics
        mean_before = np.mean(mu_before)
        mean_after = np.mean(mu_after)
        percent_change = ((mean_after - mean_before) / mean_before) * 100
        
        # Credible intervals
        ci_before = np.percentile(mu_before, [2.5, 97.5])
        ci_after = np.percentile(mu_after, [2.5, 97.5])
        
        # Find closest event
        cp_date = data['Date'].iloc[cp_idx]
        events_df['Date_dt'] = pd.to_datetime(events_df['Date'])
        closest_event = events_df.iloc[(events_df['Date_dt'] - cp_date).abs().argsort()[:1]]
        
        results.append({
            'change_point': i+1,
            'date': cp_date,
            'mean_before': mean_before,
            'mean_after': mean_after,
            'percent_change': percent_change,
            'ci_before': ci_before,
            'ci_after': ci_after,
            'closest_event': closest_event['Event Name'].values[0] if not closest_event.empty else 'Unknown',
            'event_date': closest_event['Date'].values[0] if not closest_event.empty else None
        })
    
    return pd.DataFrame(results)

In [ ]:
def correlate_events_with_changepoints(change_points_df, events_df, window_days=30):
    """
    Correlate detected change points with historical events
    """
    correlations = []
    
    for _, cp in change_points_df.iterrows():
        cp_date = cp['date']
        
        # Find events within window
        events_df['Date_dt'] = pd.to_datetime(events_df['Date'])
        time_diffs = (events_df['Date_dt'] - cp_date).abs()
        nearby_events = events_df[time_diffs.dt.days <= window_days]
        
        for _, event in nearby_events.iterrows():
            # Calculate pre/post event statistics
            event_date = event['Date_dt']
            pre_window = data[(data['Date'] >= event_date - pd.Timedelta(days=window_days)) & 
                             (data['Date'] < event_date)]
            post_window = data[(data['Date'] > event_date) & 
                              (data['Date'] <= event_date + pd.Timedelta(days=window_days))]
            
            if len(pre_window) > 0 and len(post_window) > 0:
                pre_mean = pre_window['Price'].mean()
                post_mean = post_window['Price'].mean()
                price_change = ((post_mean - pre_mean) / pre_mean) * 100
                
                correlations.append({
                    'change_point_date': cp_date,
                    'event_name': event['Event Name'],
                    'event_date': event_date,
                    'days_difference': (event_date - cp_date).days,
                    'pre_event_mean': pre_mean,
                    'post_event_mean': post_mean,
                    'price_change_percent': price_change,
                    'event_category': event['Category']
                })
    
    return pd.DataFrame(correlations)

In [ ]:
def main():
    # Load data
    print("Loading data...")
    data = load_and_prepare_data('data/raw/brent_prices.csv')
    events_df = pd.read_csv('data/processed/events_dataset.csv')
    
    # EDA
    print("Performing exploratory analysis...")
    plot_price_series(data)
    
    # Build and run model
    print("Building Bayesian change point model...")
    model = bayesian_change_point_model(data, n_changepoints=5)
    
    print("Running MCMC sampling...")
    trace = run_mcmc(model)
    
    # Analyze results
    print("Analyzing results...")
    summary = analyze_results(trace, data, events_df)
    
    # Get most probable change points
    tau_samples = trace.posterior['tau_sorted'].values
    most_probable_cps = np.median(tau_samples, axis=(0,1)).astype(int)
    
    # Quantify impacts
    print("Quantifying impacts...")
    impacts_df = quantify_impacts(trace, data, most_probable_cps)
    print("\nImpact Analysis:")
    print(impacts_df.to_string())
    
    # Correlate with events
    correlations_df = correlate_events_with_changepoints(
        impacts_df, events_df, window_days=45
    )
    
    print("\nEvent Correlations:")
    print(correlations_df.sort_values('price_change_percent', ascending=False).head(10))
    
    # Save results
    impacts_df.to_csv('results/change_point_impacts.csv', index=False)
    correlations_df.to_csv('results/event_correlations.csv', index=False)
    
    return data, trace, impacts_df, correlations_df

if __name__ == "__main__":
    data, trace, impacts, correlations = main()